# Image2Image Pipeline for Stable Diffusion using 🧨 Diffusers 

This notebook shows how to create a custom `diffusers` pipeline for  text-guided image-to-image generation with Stable Diffusion model using  🤗 Hugging Face [🧨 Diffusers library](https://github.com/huggingface/diffusers). 

For a general introduction to the Stable Diffusion model please refer to this [colab](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb).



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

In [ ]:
!pip install diffusers==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"

You need to accept the model license before downloading or using the weights. In this post we'll use model version `v1-4`, so you'll need to  visit [its card](https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree. 

You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work. For more information on access tokens, please refer to [this section of the documentation](https://huggingface.co/docs/hub/security-tokens).

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Image2Image pipeline.

In [ ]:
import inspect
import warnings
from typing import List, Optional, Union

import torch
from torch import autocast
from tqdm.auto import tqdm

from diffusers import StableDiffusionImg2ImgPipeline

Load the pipeline

In [ ]:
device = "cuda"
model_path = "CompVis/stable-diffusion-v1-4"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_path,
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=True
)
pipe = pipe.to(device)

Download an initial image and preprocess it so we can pass it to the pipeline.

In [ ]:
import requests
from io import BytesIO
from PIL import Image
from datetime import datetime

# init_image_path = '/content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/init_image/image.png'

# init_image_path = '/content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/init_image/base_frame.png'
init_image_path = '/content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/init_image/large_blue_vase.png'

# init_image_path = '/content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/init_image/5 large blue and white vases.png'
init_img = Image.open(init_image_path).convert("RGB")
init_img = init_img.resize((512, 768))
init_img

In [ ]:
### black and white option
# init_img = init_img.convert('L')
# init_img = init_img.convert('RGB')
# init_img

Define the prompt and run the pipeline.

In [ ]:
prompt = "A large blue and white baluster vase and cover, Qing dynasty, Kangxi period 1722, in the style of Van Gogh, the starry night motif"

In [ ]:
prompt ="VASE EN PORCELAINE BLEU BLANC, Chine, dynastie Qing, fin du XVIIIe siècle. De forme de balustre, à décor de fleurs et rinceaux feuillagés, la bordure du col ornée d'une frise feuilles, le pied, de flots stylisés"

In [ ]:
# prompt = "A magnificent and very rare carved green-glazed vase with intricate and very detailed flowers carvings, delicately enamelled, Christies auction, high quality picture of a vase in a white background"

In [ ]:
# prompt = "a pair of french armchairs in the style Louis XV ormolu, bright colorful floral pattern on the armchair textiles, gilded wooden armchairs, high resolution award winning photograph"

Here, `strength` is a value between 0.0 and 1.0, that controls the amount of noise that is added to the input image. Values that approach 1.0 allow for lots of variations but will also produce images that are not semantically consistent with the input.

In [ ]:
def produce_one_image_per_seed(seed):
  generator = torch.Generator(device=device).manual_seed(seed)

  now = datetime.now() 
  time = now.strftime("%H%M%S")

  with autocast("cuda"):
      pipe.safety_checker = (lambda images, clip_input: (images, False))
      image = pipe(prompt=prompt, init_image=init_img, strength=0.65, guidance_scale=7.5, generator=generator).images[0]
  image.save('/content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/output/'+time+'image.png')
  # display(image)

In [ ]:
def produce_one_image(input_strength):
  generator = torch.Generator(device=device).manual_seed(4242)

  now = datetime.now() 
  time = now.strftime("%H%M%S")

  with autocast("cuda"):
      pipe.safety_checker = (lambda images, clip_input: (images, False))
      image = pipe(prompt=prompt, init_image=init_img, strength=input_strength, guidance_scale=7.5, generator=generator).images[0]
  image.save('/content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/output/'+time+'image.png')
  # display(image)

In [ ]:
# for input_strength in range(30, 90, 5):
#   curr_coef = input_strength/100
#   print("Computing coef: "+str(curr_coef))
#   produce_one_image(curr_coef)

In [ ]:
for input_strength in range(1, 200, 1):
  curr_coef = input_strength*13
  print("Computing coef: "+str(curr_coef))
  produce_one_image_per_seed(curr_coef)

In [ ]:
# for input_strength in range(1, 20, 1):
#   curr_coef = input_strength*420
#   print("Computing coef: "+str(curr_coef))
#   produce_one_image_per_seed(curr_coef)

In [ ]:
!mkdir /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/$(date +%Y%m%d_%H%M%S) && mv /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/output/*.png /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/$(date +%Y%m%d_%H%M%S)